In [ ]:
import pandas as pd
from keras.models import load_model
import numpy as np

In [ ]:
print("Enter output filename")
output = input()
print("Enter test data filename")
file_name = input()

In [ ]:

df = pd.read_csv(file_name)

# Drop variable
df = df.drop(columns=['open', 'high', 'low', 'close', 'volume', 'dividend', 'split'])

# Split data by IDs and write to different files
dfs = dict(tuple(df.groupby('id')))
list_df = [dfs[x] for x in dfs]
for index, df in enumerate(list_df):
    df['moving_average'] = df['adjusted_close'].rolling(5).mean()
    df.to_csv("data_by_id/" + str(index) + ".csv", index=False)

In [ ]:
import os
from sklearn.preprocessing import MinMaxScaler

directory_in_str = os.getcwd()
directory_in_str += '/data_by_id'

In [ ]:
def change_category(now, week_from_now):
    ratio = float(week_from_now)/now
    percentile = (ratio - 1) * 100
    if abs(percentile) >= 5:
        return np.sign(percentile) * 3
    elif abs(percentile) >= 3:
        return np.sign(percentile) * 2
    elif abs(percentile) >= 2:
        return np.sign(percentile)
    else:
        return 0

In [ ]:
def LSTM_predict(filename):
    filename = 'data_by_id/'+filename
    print(filename)
    df = pd.read_csv(filename, index_col="time",parse_dates=True)
    df = df[4:]
    column = df['adjusted_open'].count()
    if (column > 60):
        return actual_predict(df, column)
    return None

In [ ]:
def actual_predict(df, column):
        temp = df.copy()
        training_set = df.drop(['id', 'adjusted_volume'], axis=1)
        # Feature Scaling
        sc = MinMaxScaler(feature_range = (0, 1))
        training_set_scaled = sc.fit_transform(training_set)
        training_set_scaled.shape
        # Creating a data structure with 60 timesteps and 1 output
        X_test = []
        for i in range(60, column):
            X_test.append(training_set_scaled[i-60:i, :])
        X_test = np.array(X_test)
        # Reshaping
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 5))
        regressor = load_model('model.h5')
        y = regressor.predict(X_test)
        return temp, y

In [ ]:
 def change_percent(together_df):
    together_df.rename(index=str, columns={3: "price"}, inplace = True)
    together_df = together_df.assign(actual_lag_5 = lambda x: x.price.shift(-5))
    together_df = together_df.assign(actual_percent = lambda x: (x.actual_lag_5 - x.price)/x.price)
    together_df = together_df.assign(up_two = lambda x: (x.actual_percent > 0.02).astype(int))
    together_df = together_df.assign(up_three = lambda x: (x.actual_percent > 0.03).astype(int))
    together_df = together_df.assign(up_five = lambda x: (x.actual_percent > 0.05).astype(int))
    
    together_df = together_df.assign(down_two = lambda x: (x.actual_percent < 0.02).astype(int))
    together_df = together_df.assign(down_three = lambda x: (x.actual_percent < 0.03).astype(int))
    together_df = together_df.assign(down_five = lambda x: (x.actual_percent < 0.05).astype(int))
    together_df = together_df.drop([0, 1, 2, 'price', 4, 'actual_lag_5', 'actual_percent'], axis=1)
    return together_df

In [ ]:
def write_to_file(ids, y):
    ids = ids.drop(['adjusted_close', 'adjusted_open', 'adjusted_high', 'adjusted_low', 'adjusted_volume', 'moving_average'], axis=1)
    time_index = ids[60:].index
    y.index = time_index
    df_all_cols = pd.concat((ids[60:], y), axis = 1)
    df_all_cols.to_csv(output)

In [ ]:
directory = os.fsencode(directory_in_str)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"):
        temp, y = LSTM_predict(filename)
        if y is not None:
            dfnew = pd.DataFrame(y)
            new = change_percent(dfnew)
            write_to_file(temp, new)